# ===========================================
# 1_data_exploration.ipynb
# Untersuchung der Textstile, Längenverteilung und Wortstatistiken
# ===========================================

# Zelle 1: Bibliotheken importieren und Setup
import os
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

# Stelle sicher, dass NLTK-Daten verfügbar sind (nur einmal ausführen)
nltk.download('punkt')

# Pfad zu den Rohdaten (angenommen, es handelt sich um ein CSV mit zwei Spalten: 'modern_text' und 'shakespeare_text')
RAW_DATA_DIR = "../data/raw"
data_file = os.path.join(RAW_DATA_DIR, "style_texts_raw.csv")  # Beispiel-Dateiname

# ===========================================
# Zelle 2: Daten einlesen
# ===========================================
df = pd.read_csv(data_file)
print("Anzahl Datensätze:", len(df))
display(df.head())

# Erwartete Spalten (Beispiel):
# modern_text: moderner Stil, z. B. "Hey, was geht ab?"
# shakespeare_text: stilisierter Text, z. B. "Pray, what doth thou desire?"

# ===========================================
# Zelle 3: Datenbereinigung (Basis)
# ===========================================
def clean_text(text):
    """Entfernt HTML-Tags, überflüssige Leerzeichen und trimmt den Text; konvertiert in Kleinbuchstaben."""
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip().lower()

# Wende die Bereinigung auf beide Stilspalten an
df["modern_clean"] = df["modern_text"].apply(clean_text)
df["shakespeare_clean"] = df["shakespeare_text"].apply(clean_text)

print("\nBereinigte Beispiele:")
display(df[["modern_clean", "shakespeare_clean"]].head())

# ===========================================
# Zelle 4: Textlängen berechnen
# ===========================================
# Wir messen die Länge (in Zeichen und in Wörtern) beider Stilarten
df["modern_char_len"] = df["modern_clean"].apply(len)
df["modern_word_count"] = df["modern_clean"].apply(lambda x: len(word_tokenize(x)))
df["shakespeare_char_len"] = df["shakespeare_clean"].apply(len)
df["shakespeare_word_count"] = df["shakespeare_clean"].apply(lambda x: len(word_tokenize(x)))

print("\nStatistiken zur Zeichenlänge (modern):")
display(df["modern_char_len"].describe())

print("\nStatistiken zur Wortanzahl (modern):")
display(df["modern_word_count"].describe())

print("\nStatistiken zur Zeichenlänge (Shakespeare):")
display(df["shakespeare_char_len"].describe())

print("\nStatistiken zur Wortanzahl (Shakespeare):")
display(df["shakespeare_word_count"].describe())

# ===========================================
# Zelle 5: Visualisierung der Längenverteilungen
# ===========================================
plt.figure(figsize=(10, 4))
plt.hist(df["modern_word_count"], bins=30, color="dodgerblue", alpha=0.7)
plt.xlabel("Wortanzahl (modern)")
plt.ylabel("Anzahl Texte")
plt.title("Verteilung der Wortanzahl in modernen Texten")
plt.show()

plt.figure(figsize=(10, 4))
plt.hist(df["shakespeare_word_count"], bins=30, color="indianred", alpha=0.7)
plt.xlabel("Wortanzahl (Shakespeare)")
plt.ylabel("Anzahl Texte")
plt.title("Verteilung der Wortanzahl in Shakespeare-Texten")
plt.show()

# ===========================================
# Zelle 6: Tokenstatistiken
# ===========================================
# Wir tokenisieren beide Spalten und erstellen Häufigkeitsverteilungen
def get_tokens(text):
    return word_tokenize(text)

df["modern_tokens"] = df["modern_clean"].apply(get_tokens)
df["shakespeare_tokens"] = df["shakespeare_clean"].apply(get_tokens)

# Erstelle einen Counter für beide Stile
modern_counter = Counter([token for tokens in df["modern_tokens"] for token in tokens])
shakespeare_counter = Counter([token for tokens in df["shakespeare_tokens"] for token in tokens])

print("Top 10 moderne Tokens:")
print(modern_counter.most_common(10))

print("\nTop 10 Shakespeare-Tokens:")
print(shakespeare_counter.most_common(10))

# Balkendiagramm der 10 häufigsten Tokens (modern)
modern_top10 = modern_counter.most_common(10)
tokens_mod, freq_mod = zip(*modern_top10)
plt.figure(figsize=(8,4))
plt.bar(tokens_mod, freq_mod, color="skyblue")
plt.title("Top 10 moderne Tokens")
plt.xlabel("Token")
plt.ylabel("Häufigkeit")
plt.xticks(rotation=45)
plt.show()

# ===========================================
# Zelle 7: Beispiele für Stilpaare
# ===========================================
# Wähle einige zufällige Zeilen aus, um die Unterschiede zu visualisieren
sample_styles = df.sample(5, random_state=42)
print("\nBeispielhafte Stilpaare:")
for idx, row in sample_styles.iterrows():
    print("\nModern:")
    print(row["modern_clean"])
    print("\nShakespeare:")
    print(row["shakespeare_clean"])
    print("-" * 50)

# ===========================================
# Zelle 8: Fazit & Nächste Schritte
# ===========================================
print("""
Fazit:
- Die Rohdaten umfassen zwei unterschiedliche Stile: modern und Shakespeare.
- Es gibt signifikante Unterschiede in der Wortwahl, Satzstruktur und Länge.
- Die Tokenstatistiken zeigen, welche Wörter in den jeweiligen Stilen dominieren.
Nächste Schritte:
- In 2_data_preprocessing.ipynb werden wir die Texte weiter bereinigen, tokenisieren und in Trainings-, Validierungs- und Test-Sets aufteilen.
- Anschließend wird ein Vokabular erstellt, das für den Stiltransfer genutzt wird.
""")
